# OpenCV

In [1]:
!pip install anvil-uplink

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp36-none-any.whl size=45216 sha256=abb6368ecb61f9fdf2b65abae4005a3f839c08da8ec423251a3a28fc7dff1a20
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


In [2]:
!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not avilable
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 150kB/s 
     |████████████████████████████████| 14.8MB 323kB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: albumentations 0.1.12 requires opencv-python, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Extract Data

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate
from pathlib import Path

import pandas as pd
import os
import zipfile as zf
import shutil
import re
import seaborn as sns

import pylab as pl
import cv2
import numpy as np
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
import anvil.server

anvil.server.connect("KHACDELUQKBVRYN5H4EHS4AK-VWORPTFVIXMKUNTH")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


#Testing

In [4]:
clf, classes_names, k, voc = joblib.load("/content/drive/MyDrive/trashnet.pkl")

In [5]:
import anvil.media
@anvil.server.callable
def upload(file):
  with anvil.media.TempFile(file) as filename:
    global im
    im = cv2.imread(filename)

In [6]:
import anvil.media
@anvil.server.callable
def ekstraksi():
  # with anvil.media.TempFile(file) as filename:
    # img = filename
    global des_list
    des_list = []
    sift = cv2.xfeatures2d.SIFT_create()

    # im = cv2.imread(img)
    grey = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    kpts, des = sift.detectAndCompute(grey, None)
    if len(kpts) < 1:
        no_kpts = np.zeros((1, sift.descriptorSize()), np.float32)
        des_list.append((im, no_kpts))
    else:
        des_list.append((im, des))

    imss = cv2.drawKeypoints(grey,kpts,im,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    import matplotlib.pyplot as plt
    from PIL import Image
    plt.imshow(imss)
    imgg=Image.fromarray(im, 'RGB')
    imgg.save('keypoint.jpg')

    return (anvil.media.from_file('keypoint.jpg', 'image/jpg'))

In [7]:
import anvil.media
@anvil.server.callable
def klasifikasi():
  from scipy.cluster.vq import vq    
  test_features = np.zeros((1, k), "float32")
  for i in range(1):
    words, distance = vq(des_list[i][1],voc)
    for w in words:
      test_features[i][w] += 1
  
  predictions =  [classes_names[i] for i in clf.predict(test_features)]
  hasil=str(predictions[0])
  return (hasil)

In [ ]:
# run dan biarin biar konek
anvil.server.wait_forever()